In [66]:
import gmaps
import pandas as pd
import requests
import json

# Google developer API key
from api_keys import g_key
from pprint import pprint

# Configure gmaps
gmaps.configure(api_key=g_key)

In [67]:
cw_df = pd.read_csv('City_Weather.csv')
cw_df.head()

,City,Longitude,Latitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,merauke,140.3333,-8.4667,86.76,64,100,6.69
1,yarada,83.2667,17.6500,77.32,69,75,2.30
2,marathon,-89.7668,44.9000,12.72,70,0,1.01
3,punta arenas,-70.9167,-53.1500,50.11,76,40,21.85
4,touros,-35.4608,-5.1989,76.10,79,100,10.00


In [68]:
locs = cw_df[["Latitude", "Longitude"]]
humidity = cw_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locs, weights=humidity, 
                                 dissipating=False,
                                 point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [119]:
good_df = cw_df.loc[(cw_df['Max Temperature'] > 70) & (cw_df['Max Temperature'] < 80) \
                                          & (cw_df['Wind Speed'] < 10) \
                                          & (cw_df['Cloudiness'] < 10)]
good_df

,City,Longitude,Latitude,Max Temperature,Humidity,Cloudiness,Wind Speed
42,hilo,115.3333,-33.6500,79.09,34,0,4.74
54,bluff,-81.4167,19.3667,78.01,86,3,1.01
119,muli,-40.2925,-20.3297,74.95,84,0,1.01
232,haines junction,-61.7485,12.0564,75.96,80,7,9.01
248,lubango,49.1816,11.2842,71.37,69,5,2.82
255,maceio,92.3058,20.8624,75.09,51,0,4.45
267,bosaso,2.4183,6.3654,75.92,80,0,3.60
399,jaipur hat,-1.7554,4.8845,77.41,88,1,4.23
402,novyy urengoy,-72.2660,11.7139,73.44,78,4,5.19


In [120]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location" : None,
    "radius" : 5000,
    "keyword" : "hotel",
    "key" : g_key
}


In [126]:
coords = []
hotel_info = []
for index, row in good_df.iterrows():
    params['location'] = f"{row['Latitude']},{row['Longitude']}"
    hotel = requests.get(base_url, params=params).json()
    try:
        hotel_info.append({'name' : hotel['results'][0]['name'], 'city' : row['City'].title()})
        coords.append((hotel['results'][0]['geometry']['location']['lat'],hotel['results'][0]['geometry']['location']['lng']))
    except:
        print("Missing field/result... skipping.")
coords

[(-33.6466789, 115.3453292),
 (19.338348, -81.378511),
 (-20.3156731, -40.29602550000001),
 (12.0254792, -61.75718959999999),
 (11.2679286, 49.1842457),
 (20.8481779, 92.2730957),
 (6.3577069, 2.3715477),
 (4.881883, -1.7551408),
 (11.7130861, -72.265197)]

In [128]:
hotel_info

[{'name': 'Esplanade Hotel', 'city': 'Hilo'},
 {'name': 'Sunshine Suites Resort', 'city': 'Bluff'},
 {'name': 'Hotel Golden Tulip Porto Vitória', 'city': 'Muli'},
 {'name': 'Radisson Grenada Beach Resort', 'city': 'Haines Junction'},
 {'name': 'Hotel Ramada', 'city': 'Lubango'},
 {'name': 'Central Resort Ltd.', 'city': 'Maceio'},
 {'name': 'Sun Beach Hôtel', 'city': 'Bosaso'},
 {'name': 'Best Western Plus Atlantic Hotel.', 'city': 'Jaipur Hat'},
 {'name': 'Hotel Juyasirain', 'city': 'Novyy Urengoy'}]

In [131]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""
info_box = [info_box_template.format(**hotel) for hotel in hotel_info]
markers = gmaps.marker_layer(coords, info_box_content=info_box)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [95]:
print(hotels["results"][0]["name"])
print(hotels["results"][0]["vicinity"])

Hyatt Place Boise/Downtown
1024 W Bannock St, Boise
